# Statistics generation and creation

Notebook to read simulation data, calculate statistics and aggregated statistics and save to separate csv files

In [1]:
from src.metrics import *
from src.data_loader import load_data
import matplotlib.pyplot as plt
import numpy as numpy
import seaborn as sns
import pandas as pd

In [2]:
results_pid_nominal = load_data('pid', 'nominal')
# results_pid_with_noise = load_data('pid', 'noise')
# results_pid_with_disturbances = load_data('pid', 'disturbances')

results_onoff_nominal = load_data('onoff', 'nominal')
# results_onoff_with_noise = load_data('onoff', 'noise')
# results_onoff_with_disturbances = load_data('onoff', 'disturbances')

results_fuzzy_nominal = load_data('fuzzy', 'nominal')
# results_fuzzy_with_noise = load_data('fuzzy', 'noise')
# results_fuzzy_with_disturbances = load_data('fuzzy', 'disturbances')

metrics = [steady_state_error, mean_square_error, overshoot, rise_time, settling_time, comfort_time, 
            energy_consumed,variance_after_settling, recovery_time, number_of_oscillations]

results = {
    "results_pid_nominal": results_pid_nominal,
   # "results_pid_with_noise": results_pid_with_noise,
   # "results_pid_with_disturbances": results_pid_with_disturbances,
     "results_onoff_nominal": results_onoff_nominal,
    # "results_onoff_with_noise": results_onoff_with_noise,
    # "results_onoff_with_disturbances": results_onoff_with_disturbances,
     "results_fuzzy_nominal": results_fuzzy_nominal,
    # "results_fuzzy_with_noise": results_fuzzy_with_noise,
    # "results_fuzzy_with_disturbances": results_fuzzy_with_disturbances,
}



In [3]:
def calculate_and_export_stats(scenario_results, metrics, scenario_name):
    aggregated_stats = pd.DataFrame(columns=['metric', 'mean', 'mean_ci_lower','mean_ci_upper', 'var', 'var_ci_lower', 'var_ci_upper'])
    stats_per_run = []
    long_format_records = []
    for metric in metrics:
        metric_results = ([metric(result, 'measuredTemp', 20.0, 'windowState' ) for result in scenario_results])
        
        stats_per_run.append({
            'metric': metric.__name__,
            'values': metric_results
        })

        for i, value in enumerate(metric_results):
            long_format_records.append({
                'controller': scenario_name,
                'run': i,
                'metric': metric.__name__,
                'value': value
            })


        # Calculate confidence intervals for mean and variance
        mean, mean_ci_lower, mean_ci_upper = compute_confidence_interval(metric_results)
        var, var_ci_lower, var_ci_upper = compute_variance_confidence_interval(metric_results)

        
        metric_entry ={
            'metric': metric.__name__,
            'mean': np.mean(metric_results),
            'mean_ci_lower': mean_ci_lower,
            'mean_ci_upper': mean_ci_upper,
            'var': np.var(metric_results),
            'var_ci_lower': var_ci_lower,
            'var_ci_upper': var_ci_upper 
        }
        aggregated_stats = pd.concat([aggregated_stats, pd.DataFrame([metric_entry])], ignore_index=True)

    aggregated_stats.to_csv(f"statistics/{scenario_name}_aggregated.csv", index=False)
    #stats_per_run_df = pd.DataFrame(stats_per_run)
    #stats_per_run_df.to_csv(f"statistics/{scenario_name}.csv", index=False)
    long_format_df = pd.DataFrame(long_format_records)
    long_format_df.to_csv(f"statistics/{scenario_name}_all.csv", index=False)

In [4]:
for name, scenario in results.items():
    calculate_and_export_stats(scenario, metrics, scenario_name=name)
    print(f"Stats for {name} calculated and exported.")



/tmp/ipykernel_72742/2595276995.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_stats = pd.concat([aggregated_stats, pd.DataFrame([metric_entry])], ignore_index=True)


Stats for results_pid_nominal calculated and exported.


/tmp/ipykernel_72742/2595276995.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_stats = pd.concat([aggregated_stats, pd.DataFrame([metric_entry])], ignore_index=True)


Stats for results_onoff_nominal calculated and exported.


/tmp/ipykernel_72742/2595276995.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  aggregated_stats = pd.concat([aggregated_stats, pd.DataFrame([metric_entry])], ignore_index=True)


Stats for results_fuzzy_nominal calculated and exported.
